# Setup

In [1]:
from openai import OpenAI
from openai import embeddings
from dotenv import load_dotenv
import os
import json

load_dotenv(dotenv_path=".openaidev.env")
print(os.getenv("OPENAI_API_KEY")[:20], "...")
print(os.getenv("OPENAI_ORG")[:8], "...")
print(os.getenv("OPENAI_PROJECT")[:8], "...")
# Set the API key and endpoint
api_key = os.getenv('OPENAI_API_KEY')
org = os.getenv('OPENAI_ORG')
project = os.getenv('OPENAI_PROJECT')
model = "gpt-4.1-mini"

sk-proj-eUanLfMUnbXn ...
org-8d28 ...
proj_30j ...


In [2]:
client = OpenAI(
    api_key=api_key,
    organization=org,
    project=project
)

## Responses API 

In [37]:
response = client.responses.create(
    model=model,
    input="Hello, world!"
)

print(response.output_text)

Hello! How can I assist you today?


In [39]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "Hello, world!"}
    ]
)
print(response.choices[0].message.content)


Hello! How can I assist you today?


# pydantic models

In [4]:
from pydantic import BaseModel, HttpUrl
from typing import List, Union, Optional

class InputText(BaseModel):
    type: str = "input_text"
    text: str

class InputImage(BaseModel):
    type: str = "input_image"
    image_url: HttpUrl

class InputData(BaseModel):
    role: str = "user"
    content: List[Union[InputText, InputImage]]


In [5]:

# Example usage
input_data = {
        "role": "user",
        "content": [
            {"type": "input_text", "text": "what's in this image?"},
            {
                "type": "input_image",
                "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
            },
        ],
    }


input_date_parsed = InputData.model_validate(input_data)

print(input_date_parsed)
print(type(input_date_parsed))

print(input_date_parsed.model_dump_json(indent=2))

role='user' content=[InputText(type='input_text', text="what's in this image?"), InputImage(type='input_image', image_url=HttpUrl('https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg'))]
<class '__main__.InputData'>
{
  "role": "user",
  "content": [
    {
      "type": "input_text",
      "text": "what's in this image?"
    },
    {
      "type": "input_image",
      "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    }
  ]
}


In [6]:
new_input = InputData(
    role="assistant",
    content=[
        InputText(text="This is a text response"),

    ]
)

print(new_input.model_dump_json(indent=2))

{
  "role": "assistant",
  "content": [
    {
      "type": "input_text",
      "text": "This is a text response"
    }
  ]
}


In [7]:
new_input2 = InputData(
    role="assistant",
    content=[
        InputImage(image_url="https://example.com/another-image.jpg"),
    ]
)

print(new_input2.model_dump_json(indent=2))

{
  "role": "assistant",
  "content": [
    {
      "type": "input_image",
      "image_url": "https://example.com/another-image.jpg"
    }
  ]
}


In [8]:
new_content = InputData(
    content=[
        InputText(text="what's in this image?"),
        InputImage(
            image_url="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
        ),
    ]
)


print(new_content.model_dump_json(indent=2))

{
  "role": "user",
  "content": [
    {
      "type": "input_text",
      "text": "what's in this image?"
    },
    {
      "type": "input_image",
      "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    }
  ]
}


In [9]:
input=[
    {
        "role": "user",
        "content": [

                { "type": "input_text", "text": "describe " },
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                }
        ]
    }
]
response = client.responses.create(
    model=model,
    input=input
)


In [10]:
print(response.output_text)

This image shows a wooden boardwalk path leading into a scenic field filled with tall green grass and various bushes. The path extends straight toward the horizon, becoming narrower in the distance. On either side of the pathway, the grass is lush and vibrant, suggesting a healthy, natural environment. The sky above is mostly clear with some scattered clouds, painted with soft shades of blue and white, indicating a calm and pleasant day. The overall atmosphere of the scene is peaceful and inviting, perfect for a nature walk or outdoor exploration.


In [11]:

print(response.to_json())

{
  "id": "resp_68d3301f98b88191a6f9904249c730bc08f76dc9c4c0b5a2",
  "created_at": 1758670880.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "response",
  "output": [
    {
      "id": "msg_68d33021d5dc81918b850f37bf99010608f76dc9c4c0b5a2",
      "content": [
        {
          "annotations": [],
          "text": "This image shows a wooden boardwalk path leading into a scenic field filled with tall green grass and various bushes. The path extends straight toward the horizon, becoming narrower in the distance. On either side of the pathway, the grass is lush and vibrant, suggesting a healthy, natural environment. The sky above is mostly clear with some scattered clouds, painted with soft shades of blue and white, indicating a calm and pleasant day. The overall atmosphere of the scene is peaceful and inviting, perfect for a nature walk or outdoor exploration.",
          "type": "output_text"

In [13]:
image_url = 'https://thumbs.dreamstime.com/z/worksheet-mathematical-puzzle-game-learning-mathematics-tasks-addition-preschool-children-kids-vector-186763165.jpg'


input=[
    {
        "role": "user",
        "content": [

                { "type": "input_text", "text": "solve " },
                {
                    "type": "input_image",
                    "image_url": image_url
                }
        ]
    }
]
response = client.responses.create(
    model=model,
    input=input
)


In [14]:
print(response.output_text)

Let's solve it step by step using the given equations:

1. Butterfly = 5

2. Grapes = Butterfly + 4
   Substitute Butterfly = 5:
   Grapes = 5 + 4 = 9

3. Apple = Grapes - 7
   Substitute Grapes = 9:
   Apple = 9 - 7 = 2

Answer:
Apple = 2


In [15]:
# display image from url
from IPython.display import Image
Image(url=image_url, height=300)


# Model Structured Output

In [16]:
user_input = "Extract product names and prices: apples, $4, bananas, $3, cherries, $5 qty 9"

response = client.responses.create(
    model=model,
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_input}
            ]
        }
    ]
)



print(response.output_text)

Here are the extracted product names and prices:

- Apples: $4
- Bananas: $3
- Cherries: $5 (quantity 9)


https://platform.openai.com/logs/resp_68d3303cac888196a858e970cf3ff0870e2cae486f8a9e9c 

In [17]:
system_message = "Where necessary, respond in JSON format"

response = client.responses.create(
    model=model,
    input=[
        {
            "role": "system",
            "content": [
                {"type": "input_text", "text": system_message}
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_input}
            ]
        }
    ]
)
print(response.output_text)


```json
{
  "products": [
    {
      "name": "apples",
      "price": 4
    },
    {
      "name": "bananas",
      "price": 3
    },
    {
      "name": "cherries",
      "price": 5,
      "quantity": 9
    }
  ]
}
```


https://platform.openai.com/logs/resp_68d33050c6788193be0dc2c3ae5e1faf07d35316f9fc8ec9

In [18]:
class ItemPrice(BaseModel):
    item: str
    price: float
    quantity: Optional[int]

class ItemPriceResponse(BaseModel):
    items: List[ItemPrice]

structured_output_response = client.responses.parse(
    model=model,
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_input}
            ]
        }
    ],
    text_format=ItemPriceResponse
)

item_prices = structured_output_response.output_parsed

https://platform.openai.com/logs/resp_68d3306fb14081a3830327ee11a3b59903a52f9ad0c1bb7d

In [19]:
[i for i in item_prices.items]

[ItemPrice(item='apples', price=4.0, quantity=None),
 ItemPrice(item='bananas', price=3.0, quantity=None),
 ItemPrice(item='cherries', price=5.0, quantity=9)]

In [20]:
print(structured_output_response.output_parsed.model_dump_json(indent=2))

{
  "items": [
    {
      "item": "apples",
      "price": 4.0,
      "quantity": null
    },
    {
      "item": "bananas",
      "price": 3.0,
      "quantity": null
    },
    {
      "item": "cherries",
      "price": 5.0,
      "quantity": 9
    }
  ]
}


In [25]:
structured_output_response = client.chat.completions.parse(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_input},
        # {"role": "user", "content":
        # "List 3 items with prices in a JSON array, with fields item and price."}
    ],
    response_format=ItemPriceResponse
)
print(structured_output_response.choices[0])


print(structured_output_response.choices[0].message.parsed.items)

ParsedChoice[ItemPriceResponse](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[ItemPriceResponse](content='{"items":[{"item":"apples","price":4,"quantity":null},{"item":"bananas","price":3,"quantity":null},{"item":"cherries","price":5,"quantity":9}]}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=ItemPriceResponse(items=[ItemPrice(item='apples', price=4.0, quantity=None), ItemPrice(item='bananas', price=3.0, quantity=None), ItemPrice(item='cherries', price=5.0, quantity=9)])))
[ItemPrice(item='apples', price=4.0, quantity=None), ItemPrice(item='bananas', price=3.0, quantity=None), ItemPrice(item='cherries', price=5.0, quantity=9)]


https://platform.openai.com/logs/chatcmpl-CJ6z10766cRoNBNzsAr1H3i72UhB2

In [34]:
parsed_items : ItemPriceResponse = structured_output_response.choices[0].message.parsed

print(type(parsed_items))

for item in parsed_items.items:
    print(f"{item.item}: ${item.price} (qty: {item.quantity})")

<class '__main__.ItemPriceResponse'>
apples: $4.0 (qty: None)
bananas: $3.0 (qty: None)
cherries: $5.0 (qty: 9)
